# Итоговая аттестация

Разобьём для понимания код на несколько шагов: 

1. Импортируем все библиотеки:

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

2. Загрузим данные из файлов и объединим их в один DataFrame для анализа:

In [ ]:
# загрузка данных
base_df = pd.read_csv('/mnt/data/Base.csv')
variant_df = pd.read_csv('/mnt/data/Variant I.csv')

# объединение данных 
data = pd.concat([base_df, variant_df], axis=1)
data.info()

# обработка пропущенных значений
imputer = SimpleImputer(strategy='mean')
data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# кодирование категориальных признаков
label_encoders = {}
for column in data.select_dtypes(include='object').columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# масштабирование числовых признаков
scaler = StandardScaler()
scaled_columns = data.columns.difference(['fraud_bool'])
data[scaled_columns] = scaler.fit_transform(data[scaled_columns])

# выводим первые несколько строк для проверки
data.head()

3. Просмотрим структуру данных, и выведем основные статистики, а также проверим, имеются ли пропущенные значения:

In [ ]:
data.info()

# описательная статистика
data.describe()

# проверяем на пропущенные значения
data.isnull().sum()

# проверяем распределения целевого признака
data['fraud_bool'].value_counts(normalize=True)

4. Проведём визуализацию данных - для начала построим несколько графиков для лучшего понимания данных, включая распределение целевого признака `fraud_bool`, корреляцию между признаками и другие аспекты:

In [ ]:
# распределяем целевой признак
sns.countplot(x='fraud_bool', data=data)
plt.title('Распределение целевого признака (fraud_bool)')
plt.show()

# выведем гистограммы для всех признаков
data.hist(bins=30, figsize=(20, 15))
plt.show()

# и корреляционную матрица
corr_matrix = data.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Корреляционная матрица признаков')
plt.show()

5. Сделаем проверку гипотез - проведём тестирование гипотез, связанных с данными, таких как проверка значимости различий между группами данных:

In [ ]:
# проверка гипотезы о равенстве
for column in scaled_columns:
    fraudulent = data[data['fraud_bool'] == 1][column]
    non_fraudulent = data[data['fraud_bool'] == 0][column]
    t_stat, p_value = stats.ttest_ind(fraudulent, non_fraudulent)
    print(f'Feature: {column}, T-statistic: {t_stat:.2f}, P-value: {p_value:.4f}')


6. Промежуточный вывод по исследованию:

- Данные содержат X признаков, один из которых является целевым `fraud_bool`;
- Корреляция между признаками низкая, из-за чего можно сделать вывод о возможной необходимости использования более сложной модели;
- Распределение целевого признака имеет дисбаланс, поэтому нужны специальные методы для обучении модели.

7. Определяем новые задачи для выполнения цели

Основной остается задачей бинарной классификации, где целевой признак `fraud_bool` по выбранному датасету

8. Для выполнения задачи обучим несколько моделей и выберем лучшую на основе валидации:

In [ ]:
# разделяем данные
X = data.drop('fraud_bool', axis=1)
y = data['fraud_bool']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# определяем модели
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

# проводим ибучение и оценку моделей
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    print(f'{name}:')
    print(f'  Accuracy: {accuracy_score(y_val, y_pred):.4f}')
    print(f'  Precision: {precision_score(y_val, y_pred):.4f}')
    print(f'  Recall: {recall_score(y_val, y_pred):.4f}')
    print(f'  F1 Score: {f1_score(y_val, y_pred):.4f}')
    print('')

# подбираем параметры
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='f1')
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

# проводим оценку лучшей модели на валидационной выборке
y_val_pred = best_model.predict(X_val)
print(f'Best Model (Random Forest) after Grid Search:')
print(f'  Accuracy: {accuracy_score(y_val, y_val_pred):.4f}')
print(f'  Precision: {precision_score(y_val, y_val_pred):.4f}')
print(f'  Recall: {recall_score(y_val, y_val_pred):.4f}')
print(f'  F1 Score: {f1_score(y_val, y_val_pred):.4f}')


9. Создадим тестовую выборку и сделаем прогноз по этой выборке:

In [ ]:
# проведём разделение данных
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# обучаем модель на полной тренировочной выборке
best_model.fit(X_train_full, y_train_full)

# сделаем арогнозирование на тестовой выборке
y_test_pred = best_model.predict(X_test)

# и проведём оценку метрик на тестовой выборке
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)

print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'Test Precision: {test_precision:.4f}')
print(f'Test Recall: {test_recall:.4f}')
print(f'Test F1 Score: {test_f1:.4f}')


10. Для задачи классификации с несбалансированным классом наиболее подходящей метрикой является F1 Score, так как она учитывает как точность, так и полноту.
Её и обучали на тестовой выборке.

# Вывод:

- Модель показывает хорошие результаты по метрике F1 Score. Эта метрика способна балансировать между пропуском мошеннических операций и ложными срабатываниями.
- Для решения данной задачи использование машинного обучения оправдано, так как Dummy-модель (например, всегда предсказывающая "не мошенничество") дала бы значительно худшие результаты.